In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
import numpy as np

In [ ]:
import torch

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/thesis/Dataset.csv')

<ipython-input-4-2a61b37fd56a>:2: DtypeWarning: Columns (19,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/thesis/Dataset.csv')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/thesis/Dataset.csv')

# Separate numeric and non-numeric columns
numeric_cols = ['subject_id', 'hadm_id', 'stay_id', 'temperature', 'heartrate',
                'resprate', 'o2sat', 'sbp', 'dbp', 'icd_version', 'gsn', 'ndc',
                'etccode', 'acuity']
categorical_cols = ['gender', 'race', 'arrival_transport', 'charttime', 'rhythm', 'pain',
                    'chiefcomplaint', 'icd_code', 'icd_title', 'name', 'etcdescription']

# Impute missing values with the mean for numeric columns
imputer_numeric = SimpleImputer(strategy='mean')
data[numeric_cols] = imputer_numeric.fit_transform(data[numeric_cols])

# Impute missing values with the most frequent value for categorical columns
imputer_categorical = SimpleImputer(strategy='most_frequent')
data[categorical_cols] = imputer_categorical.fit_transform(data[categorical_cols])

# Split 'charttime' into date and time components if 'charttime' is in the dataset
if 'charttime' in data.columns:
    data[['Date', 'Time']] = data['charttime'].str.split(' ', expand=True)
    data.drop(['Time', 'charttime'], axis=1, inplace=True)
    categorical_cols.append('Date')

# Separate features (X) and target (y)
X = data.drop(columns=['disposition'], errors='ignore')
y = data['disposition']

# Standardize numerical columns
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

# Convert categorical columns to category dtype and perform frequency encoding
for col in categorical_cols:
    if col in X.columns:
        freq = X[col].value_counts() / len(X)
        X[col] = X[col].map(freq)

# Ensure there are exactly 25 features
assert X.shape[1] == 25, f"Expected 25 features, but got {X.shape[1]} features."
from sklearn.preprocessing import LabelEncoder

# Convert categorical target variable to numeric if necessary
if y.dtype == 'object' or isinstance(y.iloc[0], str):
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

# Convert the data to tensors
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
X_tensor = torch.tensor(X.values, dtype=torch.float32, device=DEVICE)
y_tensor = torch.tensor(y, dtype=torch.long, device=DEVICE)

# Split the data into training and testing sets
# Split the data into training and testing sets with stratification
# Move tensors to CPU and convert to NumPy arrays
X_cpu = X_tensor.cpu().numpy()
y_cpu = y_tensor.cpu().numpy()

# Split the data into training and testing sets with stratification
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X_cpu, y_cpu, test_size=0.25, random_state=42, stratify=y_cpu)

# Convert the NumPy arrays back to PyTorch tensors and move to the appropriate device
X_train = torch.tensor(X_train_np, dtype=torch.float32, device=DEVICE)
X_test = torch.tensor(X_test_np, dtype=torch.float32, device=DEVICE)
y_train = torch.tensor(y_train_np, dtype=torch.long, device=DEVICE)
y_test = torch.tensor(y_test_np, dtype=torch.long, device=DEVICE)


# Print the shapes of the training and testing sets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Create DataLoader objects for train and test sets
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define your model
class SimpleNN(nn.Module):
    def __init__(self, input_dim, n_classes=8):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, n_classes)
        self.prelu = nn.PReLU()

    def forward(self, x):
        x = self.prelu(self.fc1(x))
        x = self.prelu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model
input_dim = X_train.shape[1]
model = SimpleNN(input_dim=input_dim, n_classes=8).to(DEVICE)

# Define hyperparameters
num_epochs = 100
learning_rate = 0.001
patience = 10

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with validation and early stopping
best_val_loss = float('inf')
patience_counter = 0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for num_features, labels in train_loader:
        num_features, labels = num_features.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(num_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}")

# Evaluation on test set
model.eval()
all_labels = []
all_preds = []
all_probs = []
correct = 0
total = 0
with torch.no_grad():
    for num_features, labels in test_loader:
        num_features, labels = num_features.to(DEVICE), labels.to(DEVICE)
        outputs = model(num_features)
        _, predicted = torch.max(outputs.data, 1)
        probs = torch.softmax(outputs, dim=1)  # Convert logits to probabilities
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())


# Convert lists to numpy arrays
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)
all_probs = np.array(all_probs)
# Compute metrics
accuracy = 100 * correct / total
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

# Binarize the labels for AUC computation
n_classes = len(np.unique(all_labels))
all_labels_bin = label_binarize(all_labels, classes=range(n_classes))

# Compute AUC-ROC score
auc = roc_auc_score(all_labels_bin, all_probs, multi_class='ovr', average='weighted')
print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"AUC-ROC Score: {auc:.2f}")



<ipython-input-5-a8227d9b7df6>:12: DtypeWarning: Columns (19,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/thesis/Dataset.csv')


X_train shape: torch.Size([786431, 25])
X_test shape: torch.Size([262144, 25])
y_train shape: torch.Size([786431])
y_test shape: torch.Size([262144])
Epoch 1, Training Loss: 0.544715896674461
Epoch 2, Training Loss: 0.517362225735269
Epoch 3, Training Loss: 0.5131107797169534
Epoch 4, Training Loss: 0.5076919211508842
Epoch 5, Training Loss: 0.5085141708210964
Epoch 6, Training Loss: 0.5099732893395412
Epoch 7, Training Loss: 0.5009954299530364
Epoch 8, Training Loss: 0.4985862902394729
Epoch 9, Training Loss: 0.5119942677774816
Epoch 10, Training Loss: 0.4948760338205223
Epoch 11, Training Loss: 0.4980785380284942
Epoch 12, Training Loss: 0.49389038362157106
Epoch 13, Training Loss: 0.49107103798451135
Epoch 14, Training Loss: 0.49769493321461294
Epoch 15, Training Loss: 0.4874610279754658
Epoch 16, Training Loss: 0.4871938691224689
Epoch 17, Training Loss: 0.48596786598742864
Epoch 18, Training Loss: 0.48409762407269835
Epoch 19, Training Loss: 0.4823084963427391
Epoch 20, Training L